In [2]:
import pandas as pd
import re

In [3]:
#reading in files
df_2012 = pd.read_csv('database_miccai_2012_with_cat.csv')
df_2021 = pd.read_csv('database_miccai_2021_with_cat.csv')

In [4]:
#removing unnecessary index column
df_2012 = df_2012.drop("Unnamed: 0", axis = 1)
df_2021 = df_2021.drop("Unnamed: 0", axis = 1)

In [5]:
#finding the references
def find_reference(year, place):
    reference_list = []
    with open(place, "r", encoding = 'utf-8') as part:
        article = part.read()
        if year == 2012:
            references = [i.start() for i in re.finditer("References\n\n1", article)]
            other_ref = [i.start() for i in re.finditer("References\n\n\[1\]", article)]
            all_ref = references + other_ref
            all_ref.sort()
            
        if year == 2021:
            all_ref = [i.start() for i in re.finditer("References", article)]
        for index in all_ref: 
            reference_list.append(article[index:index+8000])

    return reference_list


In [6]:
#saving the references
ref_list2012 = find_reference(2012, r"C:\Users\chris\Desktop\Value-Analysis-Thesis\combining proceedings txt\miccai_2012_full_txt.txt")
ref_list2021 = find_reference(2021, r"C:\Users\chris\Desktop\Value-Analysis-Thesis\combining proceedings txt\miccai_2021_full_txt.txt")

In [7]:
def create_ref_dic(ref_list):
    year={}
    i=0
    for element in ref_list:
        year[i] = []
        references = [i.start() for i in re.finditer("\([0-9][0-9][0-9][0-9]\)", element)]
        for index in references:
            num = element[index+1:index+5]
            if int(num) < 2023: #eliminating some mistakes with numbers that are clearly not years
                year.get(i).append(num)
        i+=1
    return year


In [8]:
def add_references(df, ref_list_all):
    
    
    row_list = df[df['category'] == 'category: classification'].index
    df_class = df[df['category'] == 'category: classification']
    
    ref_dic = create_ref_dic(ref_list_all)
    
    oldest_list = []
    newest_list = []
    range_list = []
    class_dic = {}
    num_ref = []

    for element in row_list:
        ref_dic.get(element).sort()
        class_dic[element] = ref_dic.get(element) #need this otherwise the sort is forgotten
        ref_list = class_dic.get(element)
        oldest_list.append(int(ref_list[0]))
        newest_list.append(int(ref_list[len(ref_list)-1]))
        range_list.append(int(ref_list[len(ref_list)-1])- int(ref_list[0]))
        num_ref.append(len(ref_list))
   
    
    data = {'index': row_list,
            'Number of references': num_ref,
            'Oldest reference': oldest_list,
            'Newest reference' : newest_list,
            'Range of references': range_list
    } 
    
    ref = pd.DataFrame(data)
    
    df_class = df_class.reset_index()
    
    df_class = df_class.merge(ref, on = 'index')
 
    
    return df_class 

    

In [9]:
df_2012class = add_references(df_2012, ref_list2012)

df_2021class = add_references(df_2021, ref_list2021)

In [98]:
#saving updated database to csv
df_2012class.to_csv("database_miccai_2012_with_ref.csv")
df_2021class.to_csv("database_miccai_2021_with_ref.csv")

In [10]:
df_2012class

,index,Title,Authors,Page numbers,DOI,Year of publication,Part of publication,category,Number of references,Oldest reference,Newest reference,Range of references
0,4,Surgical Gesture Classification from Video Data,"Benjamín Béjar Haro, Luca Zappella, René Vidal",34-41,/chapter/10.1007/978-3-642-33415-3_5,2012.0,1,category: classification,26,1999,2012,13
1,7,Efficient Optic Cup Detection from Intra-image...,"Yanwu Xu, Jiang Liu, Stephen Lin, Dong Xu, Car...",58-65,/chapter/10.1007/978-3-642-33415-3_8,2012.0,1,category: classification,16,1992,2011,19
2,9,Thoracic Abnormality Detection with Data Adapt...,"Yang Song, Weidong Cai, Yun Zhou, Dagan Feng",74-81,/chapter/10.1007/978-3-642-33415-3_10,2012.0,1,category: classification,12,2004,2011,7
3,10,Domain Transfer Learning for MCI Conversion Pr...,"Bo Cheng, Daoqiang Zhang, Dinggang Shen",82-90,/chapter/10.1007/978-3-642-33415-3_11,2012.0,1,category: classification,15,2001,2012,11
4,12,Incremental Kernel Ridge Regression for the Pr...,"Binbin Pan, James J. Xia, Peng Yuan, Jaime Gat...",99-106,/chapter/10.1007/978-3-642-33415-3_13,2012.0,1,category: classification,11,1985,2009,24
...,...,...,...,...,...,...,...,...,...,...,...,...
86,241,Patient-Adaptive Lesion Metabolism Analysis by...,"Fei Gao, Huafeng Liu, Pengcheng Shi",558-565,/chapter/10.1007/978-3-642-33454-2_69,2012.0,3,category: classification,12,1998,2011,13
87,242,A Personalized Biomechanical Model for Respira...,"B. Fuerst, T. Mansi, Jianwen Zhang, P. Khurd, ...",566-573,/chapter/10.1007/978-3-642-33454-2_70,2012.0,3,category: classification,13,2004,2011,7
88,243,Endoscope Distortion Correction Does Not (Easi...,"Jutta Hämmerle-Uhl, Yvonne Höller, Andreas Uhl...",574-581,/chapter/10.1007/978-3-642-33454-2_71,2012.0,3,category: classification,12,1973,2012,39
89,247,An Invariant Shape Representation Using the An...,"A. A. Joshi, S. Ashrafulla, D. W. Shattuck, H....",607-614,/chapter/10.1007/978-3-642-33454-2_75,2012.0,3,category: classification,19,1976,2011,35
